## Preprocessing

In [243]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [244]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_drop = application_df.drop(["EIN", "NAME"],axis=1)
application_df_drop.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [245]:
# Determine the number of unique values in each column.
application_df_drop.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [246]:
# Determine the values counts for the Application Type
bin_application_type = application_df_drop

application_type_values = bin_application_type['APPLICATION_TYPE'].value_counts()
print(application_type_values)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [247]:
# Need to bin the low application type count items into a common type "OTHER".  
application_types_to_replace = list(application_type_values[application_type_values < 1000].index)

for applications in application_types_to_replace:
    bin_application_type['APPLICATION_TYPE'] = bin_application_type['APPLICATION_TYPE'].replace(applications, "OTHER")

In [248]:
# Verify the low count application types have been binned into the "OTHER" category
bin_application_type['APPLICATION_TYPE'].value_counts()

T3       27037
OTHER     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [249]:
# Look at CLASSIFICATION value counts for binning
bin_classification = bin_application_type

classification_values = bin_classification['CLASSIFICATION'].value_counts()
print(classification_values)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [250]:
# You may find it helpful to look at CLASSIFICATION value counts >1
bin_classification_alt = classification_values[classification_values > 1]
bin_classification_alt

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [251]:
# Choose a cutoff value and create a list of classifications to be replaced
classifications_to_replace = list(classification_values[classification_values < 1000].index)

for classification in classifications_to_replace:
    bin_classification['CLASSIFICATION'] = bin_classification['CLASSIFICATION'].replace(classification, "OTHER")

In [252]:
# Check to make sure binning was successful
bin_classification['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
OTHER     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [253]:
# Optimization - dropping additional columns
# bin_classification = bin_classification.drop(['STATUS'],axis=1)
# bin_classification = bin_classification.drop(['SPECIAL_CONSIDERATIONS'], axis=1)

In [254]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_dummies = pd.get_dummies(bin_classification)
application_df_dummies

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_OTHER,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [255]:
# Split our preprocessed data into our features and target arrays
y = application_df_dummies["IS_SUCCESSFUL"].values
X = application_df_dummies.drop(["IS_SUCCESSFUL"],axis=1).values

In [256]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=100)

In [257]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [258]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=40))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_59 (Dense)             (None, 16)                656       
_________________________________________________________________
dense_60 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_61 (Dense)             (None, 1)                 17        
Total params: 945
Trainable params: 945
Non-trainable params: 0
_________________________________________________________________


In [259]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [260]:
# Train the model
#  YOUR CODE GOES HERE

os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

cp_callback = ModelCheckpoint(
    filepath = checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_frequency=1000)

fit_model = nn.fit(X_train_scaled, y_train, epochs=50, callbacks=[cp_callback])

Epoch 1/50
767/804 [===========================>..] - ETA: 0s - loss: 0.6066 - accuracy: 0.6962
Epoch 00001: saving model to checkpoints\weights.01.hdf5
804/804 [==============================] - 1s 812us/step - loss: 0.6052 - accuracy: 0.6969
Epoch 2/50
743/804 [==========================>...] - ETA: 0s - loss: 0.5645 - accuracy: 0.7257
Epoch 00002: saving model to checkpoints\weights.02.hdf5
804/804 [==============================] - 1s 750us/step - loss: 0.5638 - accuracy: 0.7264
Epoch 3/50
790/804 [============================>.] - ETA: 0s - loss: 0.5582 - accuracy: 0.7289
Epoch 00003: saving model to checkpoints\weights.03.hdf5
804/804 [==============================] - 1s 720us/step - loss: 0.5587 - accuracy: 0.7284
Epoch 4/50
791/804 [============================>.] - ETA: 0s - loss: 0.5562 - accuracy: 0.7308
Epoch 00004: saving model to checkpoints\weights.04.hdf5
804/804 [==============================] - 1s 719us/step - loss: 0.5561 - accuracy: 0.7310
Epoch 5/50
744/804 [====

In [261]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5603 - accuracy: 0.7254
Loss: 0.5602837800979614, Accuracy: 0.7253644466400146


In [262]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE
nn.save("AlphabetSoupCharity_Optimization.h5")